In [52]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

from tqdm import tqdm

In [53]:
DEBUG = True
DEVICE = torch.device('cuda:0') if torch.cuda.is_available() and not DEBUG else 'cpu'

In [54]:
class CNNModel(nn.Module):
    def __init__(self, device='cpu'):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 3, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(3, 8, 2)
        self.linear1 = nn.Linear(8 * 2 * 3, 8 * 3)
        self.linear2 = nn.Linear(24, 8)
        self.linear3 = nn.Linear(8, 1)
        self.to(device)


    def forward(self, x):
        x = self.conv1(x)
        # print("Layer 1 output shape: " + str(x.shape))
        x = self.pool1(x)
        # print("Pool 1 output shape: " + str(x.shape))
        x = self.conv2(x)
        # print("Layer 2 output shape: " + str(x.shape))
        x = torch.flatten(x, 1) # flatten
        # print("Flattened shape: " + str(x.shape))
        x = self.linear1(x)
        # print("Linear 1 output shape: " + str(x.shape))
        x = self.linear2(x)
        # print("Linear 2 output shape: " + str(x.shape))
        x = self.linear3(x)
        # print("Linear 3 output shape: " + str(x.shape))
        x = torch.sigmoid(x)
        return x

In [55]:
import DataImport
INPUT_FILE = "train.csv"

print("Importing Data...")
x_train, y_train = DataImport.import_data(INPUT_FILE, DEVICE)
model = CNNModel()
print("input shape " + str(x_train.shape))
test_output = model(x_train)
print("output shape: " + str(test_output.shape))
# print(test_output)

Importing Data...


100%|██████████| 70312/70312 [07:14<00:00, 161.86it/s] 


input shape torch.Size([70322, 1, 11, 13])
output shape: torch.Size([70322, 1])


In [56]:
def train(model, x_train, y_train, n_epochs=5, lr=0.0001):
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    losses = []
    for epoch in tqdm(range(n_epochs)):
        optimizer.zero_grad()

        output_train = model(x_train)
        # print(output_train)
        loss_train = criterion(output_train, y_train)
        loss_train.backward()
        optimizer.step()
        loss_value = loss_train.item()
        losses.append(loss_value)
        if epoch % 100 == 0:
            print("Epoch {}: loss = {}".format(str(epoch), str(loss_value)))

        # if len(output_train.nonzero()) == 0:        # not sure if this works
        #     print("Epoch {}: all zero output!".format(str(epoch)))

    return losses

def calc_acc(y_pred, y, th=1.0):
    #tp = 0
    #tn = 0
    size = len(y)
    #for i in range(size):
    #    if y_pred[i] >= th and y[i] == 1:
    #        tp += 1
    #    elif y_pred[i] < th and y[i] == 0:
    #        tn += 1
    #return float(tp + tn)*100/size
    with torch.no_grad():
        #print(y)
        #print(y_pred)
        y_th = torch.heaviside(y_pred - torch.tensor(th).expand_as(y_pred), torch.tensor(1.0).expand_as(y_pred))
        tp = torch.sum(torch.logical_and(y_th, y))
        p_pred = torch.sum(y_th)
        p_label = torch.sum(y)
        #print(y_th)
        #print(torch.logical_xor(y_th, y))
        acc = float(size - torch.sum(torch.logical_xor(y_th, y)))*100/size
        prec = float(tp)*100/p_pred
        rec = float(tp)*100/p_label
    return [acc, prec, rec]

In [57]:
model = CNNModel(DEVICE)
losses = train(model, x_train, y_train, n_epochs=1000, lr=0.0002)
# print(losses)

  0%|          | 1/1000 [00:00<11:28,  1.45it/s]

Epoch 0: loss = 1.4922934770584106


 10%|█         | 101/1000 [01:04<09:30,  1.58it/s]

Epoch 100: loss = 0.7421825528144836


 20%|██        | 201/1000 [02:10<08:21,  1.59it/s]

...

 80%|████████  | 801/1000 [08:48<02:18,  1.43it/s]

Epoch 800: loss = 0.7237185835838318


 90%|█████████ | 901/1000 [09:53<01:05,  1.51it/s]

Epoch 900: loss = 0.7234734892845154


100%|██████████| 1000/1000 [10:57<00:00,  1.52it/s]


In [58]:
with torch.no_grad():
    y_prediction = model(x_train)
    evauluation_result = calc_acc(y_prediction, y_train)
    print(evauluation_result)

[48.21250817667302, tensor(62.9630), tensor(0.0467)]
